In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scienceplots

plt.style.use(['science', 'bmh', 'notebook', 'grid'])

# fmane = "J:\\Universidade\\Projetos-UFF\\tomi.csv"
fmane2 = "./uploads/@CN_LONGA.csv"

# df = pd.read_csv(fmane,
#                  skiprows=0,
#                  delimiter=';')

# df = pd.read_csv('./uploads/tomi.csv',
#                  skiprows=0,
#                  delimiter=';')

dataframe = pd.read_csv(fmane2,
                 skiprows=0,
                 delimiter=';',
                 usecols=(0,1,2,3,4,5,6))

profundidade = dataframe.iloc[:,0]
TVDSS = dataframe.iloc[:,1]
TN_perc = dataframe.iloc[:,2]
d15N = dataframe.iloc[:,3]
TOC_perc = dataframe.iloc[:,4]
d13C = dataframe.iloc[:,5]
TOC_TN = dataframe.iloc[:,6]

In [17]:
import numpy as np
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt

# Dados dos pontos-chave de probabilidade
art_toc_tn = np.array([4,4,4,4,10,10,10,10,100,100,100,100])  # Lista/array dos valores de TOC:TN dos pontos-chave
art_c13corg = np.array([-10,-22,-25,-34,-10,-22,-25,-34,-10,-22,-25,-34])  # Lista/array dos valores de δ13Corg dos pontos-chave
probabilidade = np.array([0,10,20,30,20,30,40,50,90,95,98,100])  # Lista/array dos valores de probabilidade dos pontos-chave

# Dados das suas amostras
amostra_art_toc_tn = TOC_TN  # Lista/array dos valores de TOC:TN das suas amostras
amostra_art_c13corg = d13C  # Lista/array dos valores de δ13Corg das suas amostras

# Grade de valores para interpolação
art_toc_tn_grid = np.linspace(0, 100, 1001)  # Valores de TOC:TN para a grade
art_c13corg_grid = np.linspace(-34, -10, 241)  # Valores de δ13Corg para a grade

# Kriging
OK = OrdinaryKriging(art_toc_tn, art_c13corg, 
                     probabilidade, variogram_model='gaussian',)
z, ss = OK.execute('grid', art_toc_tn_grid, art_c13corg_grid)

# Aplicação dos dados das suas amostras na grade interpolada
amostra_probabilidade, ss = OK.execute('points', 
                                       amostra_art_toc_tn, 
                                       amostra_art_c13corg)

print(np.shape(z))
print(np.shape(amostra_probabilidade))

# scatter = plt.scatter(amostra_art_toc_tn, amostra_art_c13corg, c=amostra_probabilidade, cmap='viridis', s=30, label='Artigo')
# plt.xlabel('TOC:TN')
# plt.ylabel('delta13C')
# # plt.xlim(0,1)
# plt.ylim(-34,-20)
# cbar = plt.colorbar(scatter)
# cbar.set_label('tomi index')
# plt.show()

(241, 1001)
(32,)


In [ ]:
plt.figure(figsize=(4,8))

plt.plot(amostra_probabilidade,profundidade, 's-',label='TOMI')
plt.gca().invert_yaxis()
plt.xlim(0,100)
plt.title("TOMI Index")
plt.xlabel("Index (%)")